In [3]:
# Install the Antigranular package
!pip install antigranular &> /dev/null

In [4]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Global Economic Forecast Hackathon with Texas A&M Aggie Data Science")

Dataset "Statistical Performance Indicators" loaded to the kernel as statistical_performance_indicators
Key Name                       Value Type     
---------------------------------------------
train_x                        PrivateDataFrame
train_y                        PrivateDataFrame
test_x                         DataFrame      

Connected to Antigranular server session id: e89834fc-a3fc-4667-97bf-f814e2baf639, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


In [5]:
%%ag
x_train = statistical_performance_indicators["train_x"]
y_train = statistical_performance_indicators["train_y"]
x_test = statistical_performance_indicators["test_x"]

In [6]:
%%ag
x_train.info()

+----+-----------------+-------------+---------------+---------+----------+
|    | Column          | numerical   | categorical   | dtype   | bounds   |
|----+-----------------+-------------+---------------+---------+----------|
|  0 | intl_org_use    | True        | False         | float64 | (0, 1)   |
|  1 | social_stats    | True        | False         | float64 | (0, 1)   |
|  2 | economic_stats  | True        | False         | float64 | (0, 1)   |
|  3 | inst_stats      | True        | False         | float64 | (0, 1)   |
|  4 | pov_ratio       | True        | False         | float64 | (0, 1)   |
|  5 | child_mortality | True        | False         | float64 | (0, 1)   |
|  6 | debt_service    | True        | False         | float64 | (0, 1)   |
|  7 | safe_water      | True        | False         | float64 | (0, 1)   |
|  8 | labor_force     | True        | False         | float64 | (0, 1)   |
|  9 | no_poverty      | True        | False         | float64 | (0, 1)   |
| 10 | zero_

In [7]:
%%ag
y_train.info()

+----+----------+-------------+---------------+---------+----------+
|    | Column   | numerical   | categorical   | dtype   | bounds   |
|----+----------+-------------+---------------+---------+----------|
|  0 | income   | True        | False         | int64   | (0, 1)   |
+----+----------+-------------+---------------+---------+----------+



In [8]:
%%ag
ag_print(x_train.metadata)

{'intl_org_use': (0, 1), 'social_stats': (0, 1), 'economic_stats': (0, 1), 'inst_stats': (0, 1), 'pov_ratio': (0, 1), 'child_mortality': (0, 1), 'debt_service': (0, 1), 'safe_water': (0, 1), 'labor_force': (0, 1), 'no_poverty': (0, 1), 'zero_hunger': (0, 1), 'good_health': (0, 1), 'quality_edu': (0, 1), 'gender_eq': (0, 1), 'clean_water': (0, 1), 'clean_energy': (0, 1), 'decent_work': (0, 1), 'innovation': (0, 1), 'reduced_ineq': (0, 1), 'cities': (0, 1), 'consump_prod': (0, 1), 'life_land': (0, 1), 'peace_justice': (0, 1), 'partnerships': (0, 1)}



In [ ]:
%%ag
import tensorflow as tf
from op_pandas import standard_scaler, PrivateDataFrame
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization,LeakyReLU
from op_tensorflow import PrivateKerasModel, PrivateDataLoader
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import L1
from tensorflow.keras.optimizers import Adadelta



seqM1 = Sequential([
    Dense(128, input_shape=(24,)),
    LeakyReLU(alpha=0.01),  # Leaky ReLU for robustness
    Dropout(0.3),
    Dense(64),
    LeakyReLU(alpha=0.01),
    Dropout(0.3),
    Dense(32),
    LeakyReLU(alpha=0.01),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Sigmoid for binary output
])




# Create DP keras model
dp_model1 = PrivateKerasModel(model=seqM1, l2_norm_clip=1, noise_multiplier=1)

# Use a standard (non-DP) optimizer directly from keras.
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# PrivateKerasModel uses similar API as standard Keras
dp_model1.compile(
	optimizer = optimizer,
	loss = 'binary_crossentropy',
	metrics = ["accuracy"]
)


/usr/local/lib/python3.10/site-packages/tensorflow/__init__.py:30: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils as _distutils
2024-12-15 14:49:22.331508: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.

2024-12-15 14:49:22.334962: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-15 14:49:22.526339: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-15 14:49:22.526394: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unabl

In [30]:
%%ag
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)

# PrivateKerasModel uses similar API as standard Keras
dp_model1.compile(
	optimizer = optimizer,
	loss = 'binary_crossentropy',
	metrics = ["accuracy"]
)

In [ ]:
%%ag
data_loader = PrivateDataLoader(feature_df=x_train , label_df=y_train, batch_size=4)

In [121]:
%%ag
from op_tensorflow import get_privacy_budget
get_privacy_budget(
    sample_size=2000,
    batch_size=32,
    num_epochs=500,
    noise_multiplier=1,
    target_delta=1,
)

=> EPSILON_REQUIRED = 0 using TARGET_DELTA = 1
Training parameters used :-
    SAMPLE_SIZE = 2000
    BATCH_SIZE = 32
    NUM_EPOCHS = 500
    NOISE_MULTIPLIER = 1
    



In [ ]:
%%ag
# Fit the model with validation split
dp_model1.fit(
    x=x_train_scaled,
    y=y_train,
    epochs=500,
    batch_size=32,
    target_delta=1
)

Epoch 1/500

115/115 - 6s - loss: 0.1330 - accuracy: 0.9462 - 6s/epoch - 50ms/step

Epoch 2/500

115/115 - 2s - loss: 0.1509 - accuracy: 0.9415 - 2s/epoch - 19ms/step

Epoch 3/500

115/115 - 2s - loss: 0.1368 - accuracy: 0.9415 - 2s/epoch - 19ms/step

Epoch 4/500

115/115 - 2s - loss: 0.1395 - accuracy: 0.9439 - 2s/epoch - 20ms/step

Epoch 5/500

115/115 - 2s - loss: 0.1327 - accuracy: 0.9434 - 2s/epoch - 19ms/step

Epoch 6/500

115/115 - 2s - loss: 0.1330 - accuracy: 0.9480 - 2s/epoch - 18ms/step

Epoch 7/500

115/115 - 2s - loss: 0.1319 - accuracy: 0.9500 - 2s/epoch - 19ms/step

Epoch 8/500

115/115 - 2s - loss: 0.1427 - accuracy: 0.9420 - 2s/epoch - 19ms/step

Epoch 9/500

115/115 - 2s - loss: 0.1320 - accuracy: 0.9501 - 2s/epoch - 20ms/step

Epoch 10/500

115/115 - 2s - loss: 0.1344 - accuracy: 0.9459 - 2s/epoch - 20ms/step

Epoch 11/500

115/115 - 2s - loss: 0.1467 - accuracy: 0.9365 - 2s/epoch - 20ms/step

Epoch 12/500

115/115 - 2s - loss: 0.1494 - accuracy: 0.9416 - 2s/epoch - 

In [ ]:
%%ag
# Fit the model with validation split
dp_model1.fit(
    x=x_train_scaled,
    y=y_train,
    epochs=500,
    batch_size=32,
    target_delta=1
)

Epoch 1/500

115/115 - 2s - loss: 0.1065 - accuracy: 0.9577 - 2s/epoch - 17ms/step

Epoch 2/500

115/115 - 2s - loss: 0.1176 - accuracy: 0.9513 - 2s/epoch - 17ms/step

Epoch 3/500

115/115 - 2s - loss: 0.1149 - accuracy: 0.9559 - 2s/epoch - 17ms/step

Epoch 4/500

115/115 - 2s - loss: 0.1031 - accuracy: 0.9592 - 2s/epoch - 17ms/step

Epoch 5/500

115/115 - 2s - loss: 0.1118 - accuracy: 0.9546 - 2s/epoch - 17ms/step

Epoch 6/500

115/115 - 2s - loss: 0.1134 - accuracy: 0.9549 - 2s/epoch - 17ms/step

Epoch 7/500

115/115 - 2s - loss: 0.1044 - accuracy: 0.9594 - 2s/epoch - 17ms/step

Epoch 8/500

115/115 - 2s - loss: 0.1115 - accuracy: 0.9546 - 2s/epoch - 17ms/step

Epoch 9/500

115/115 - 2s - loss: 0.1033 - accuracy: 0.9571 - 2s/epoch - 17ms/step

Epoch 10/500

115/115 - 2s - loss: 0.1061 - accuracy: 0.9577 - 2s/epoch - 17ms/step

Epoch 11/500

115/115 - 2s - loss: 0.1142 - accuracy: 0.9577 - 2s/epoch - 17ms/step

Epoch 12/500

115/115 - 2s - loss: 0.1010 - accuracy: 0.9623 - 2s/epoch - 

In [32]:
%%ag
y_pred1 = dp_model1.predict(PrivateDataFrame(x_test_scaled), label_columns=["output"])

13/13 [==============================] - 0s 8ms/step



In [33]:
%%ag
# Note that the predictions are a float scalar
# so we scale it
def f(x: float) -> float:
  if x > 0.5:
    return 1
  else:
    return 0

y_pred1["output"] = y_pred1["output"].map(f, output_bounds=(0, 1))


In [34]:
%%ag
result = submit_predictions(y_pred1)

score: {'leaderboard': 0.9323538132799825, 'logs': {'BIN_ACC': 0.9358346463632597, 'LIN_EPS': -0.0034808330832773356}}



In [127]:

session.privacy_odometer()

{'total_epsilon_used': 0.6961666166554671,
 'total_delta_used': 0.0002,
 'library_costs': {'op_tensorflow': {'total_delta': 0.0002,
   'total_epsilon': 0.6961666166554671,
   'total_requests': 20}},
 'dataset_costs': {'Statistical Performance Indicators': {'delta': 0.0002,
   'eps': 0.6961666166554671}}}

In [ ]:
session.terminate_session()

{'status': 'ok'}